<a href="https://colab.research.google.com/github/pj2111/Assignments/blob/master/assignment_data/NLP_Practice/warm_up_model_inferencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [3]:
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk
from rich import print

In [4]:
dataset_name = "Hypersniper/riddles_v1"
dataset= load_dataset(dataset_name, split='all')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
dataset

Dataset({
    features: ['answer', 'output', 'instruction'],
    num_rows: 469
})

In [6]:
dataset[0]

{'answer': 'music',
 'output': "The question seems to be a riddle that is focusing on an entity that can evoke a range of emotional responses. The given clues are that it can make someone dream or stamp their feet, and it can also make someone laugh or weep. \n\nLet's break down these clues. Dreaming and stamping feet are both reactions that can be associated with feelings of joy, excitement, or anticipation. On the other hand, laughing and weeping are expressions of happiness and sadness respectively. So, we are looking for something that can induce these varied emotional reactions.\n\nConsidering these clues, one plausible answer could be 'music'. Here's why: \n\nMusic has a profound impact on our emotions. It has the power to uplift our spirits, soothe our nerves, make us feel happy, sad, excited, calm, and even lead us into a state of introspection or dreaming. \n\nWhen we listen to upbeat music, it often makes us want to move or dance. Hence, the reference to 'stamp their feet'. I

In [7]:
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig)
import torch

In [8]:
model_path = "roberta-large"

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
!pip install accelerate

In [10]:
roberta = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto"
)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [12]:
roberta_tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [16]:
roberta_tokenizer.default_chat_template

"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [18]:
import random
questions = [q for q in dataset['instruction']]
random.shuffle(questions)

prompt_template = """Below is a riddle. Come up with 10 more.
Output just the riddles. No numbering and don't output anything else."""

simple_request = "Generate 10 riddles that you know"

prompt_riddles = prompt_template + "\n\n".join(questions[0:10])

message_to_model = [{"role":"user", "content":prompt_riddles}]

simple_message = [{"role":"user", "content":simple_request}]

In [20]:
roberta_input = roberta_tokenizer.apply_chat_template(simple_message,
                                                      return_tensors='pt')#.to('cuda')
roberta_input

tensor([[41552, 15483,   757,  1215, 13124, 15483, 15698, 12105, 50118, 40025,
           877,   158,   910, 40741,    14,    47,   216, 41552, 15483,   757,
          1215,  1397, 15483, 15698, 50118]])

In [22]:
roberta_tokenizer.decode(roberta_input[0])

'<|im_start|>user\nGenerate 10 riddles that you know<|im_end|>\n'

In [ ]:
roberta